In [1]:
def readfile(filename):
    file = open(filename, 'rt')
    original_text = file.read()
    file.close()
    
    return original_text

In [2]:
def writefile(filename, content):
    file = open(filename, 'w')
    file.write(str(content))
    file.close()

In [3]:
def preprocess_data(text):
    # remove punctuations !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    punctuation_list = ["!","\"","#","$","%","&","'","(",")","*","+",",","-",".","/",":",";","<","=",">","?","@","[","\\","]","^","_","`","{","|","}","~","–","€","•","«","»","’","“","”","£"]
    for punctuation in punctuation_list:
        text = text.replace(punctuation, "")

    # remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    text = RE_EMOJI.sub(r'', text)

    # substitute multiple spaces with a single space
    text = re.sub(' +', ' ', text)

    # normalize text to lower case
    text = text.lower()
    
    return text

In [4]:
import re

text = readfile('train.txt')

text = text.replace("\n", " ")

text = preprocess_data(text)

writefile("cleaned_train.txt", text)

In [5]:
keyword_pair_dict = {
    "a|á": { "keywords": ["a","á"], "probability": 0 },
    "ais|áis": { "keywords": ["ais","áis"], "probability": 0 },
    "aisti|aistí": { "keywords": ["aisti","aistí"], "probability": 0 },
    "ait|áit": { "keywords": ["ait","áit"], "probability": 0 },
    "ar|ár": { "keywords": ["ar","ár"], "probability": 0 },
    "arsa|ársa": { "keywords": ["arsa","ársa"], "probability": 0 },
    "ban|bán": { "keywords": ["ban","bán"], "probability": 0 },
    "cead|céad": { "keywords": ["cead","céad"], "probability": 0 },
    "chas|chás": { "keywords": ["chas","chás"], "probability": 0 },
    "chuig|chúig": { "keywords": ["chuig","chúig"], "probability": 0 },
    "dar|dár": { "keywords": ["dar","dár"], "probability": 0 },
    "do|dó": { "keywords": ["do","dó"], "probability": 0 },
    "gaire|gáire": { "keywords": ["gaire","gáire"], "probability": 0 },
    "i|í": { "keywords": ["i","í"], "probability": 0 },
    "inar|inár": { "keywords": ["inar","inár"], "probability": 0 },
    "leacht|léacht": { "keywords": ["leacht","léacht"], "probability": 0 },
    "leas|léas": { "keywords": ["leas","léas"], "probability": 0 },
    "mo|mó": { "keywords": ["mo","mó"], "probability": 0 },
    "na|ná": { "keywords": ["na","ná"], "probability": 0 },
    "os|ós": { "keywords": ["os","ós"], "probability": 0 },
    "re|ré": { "keywords": ["re","ré"], "probability": 0 },
    "scor|scór": { "keywords": ["scor","scór"], "probability": 0 },
    "te|té": { "keywords": ["te","té"], "probability": 0 },
    "teann|téann": { "keywords": ["teann","téann"], "probability": 0 },
    "thoir|thóir": { "keywords": ["thoir","thóir"], "probability": 0 },
}

In [6]:
word_count_dict = {}
# word_count_dict_builtin = {}
words = text.split()

num_all_words = len(words)

for word in words:
    if word in word_count_dict:
        word_count_dict[word] += 1
    else:
        word_count_dict[word] = 1
        
for key in keyword_pair_dict:
    keyword1 = keyword_pair_dict[key]["keywords"][0]
    keyword2 = keyword_pair_dict[key]["keywords"][1]
    
#     word_count_dict_builtin[keyword1] = text.count(keyword1)
#     word_count_dict_builtin[keyword2] = text.count(keyword2)

# print(word_count_dict_builtin["thoir"])
# print(word_count_dict["thoir"])

#  P(thoir) = C(thoir) / [C(thoir) + C(thòir)]

In [7]:
# words = ['I','love','you']

# count bigram
for i in range(len(words)):
    
    if i == 0:
        # a word with nothing preceding
        preceding_word = "^";
    else:
         # normal pairs of words   
        preceding_word = words[i-1];
    
    bigram = preceding_word + " " + words[i]
    if bigram in word_count_dict:
        word_count_dict[bigram] += 1
    else:
        word_count_dict[bigram] = 1

# count when the word is the last word of the sentence
bigram = words[-1] + "$"
if bigram in word_count_dict:
    word_count_dict[bigram] += 1
else:
    word_count_dict[bigram] = 1



In [8]:
print(word_count_dict["thóir"] / (word_count_dict["thóir"] + word_count_dict["thoir"]))

0.5086159985488845


In [9]:
# word_1 = keyword_pair_map[0][0];
# word_2 = keyword_pair_map[0][1];
# prob_word_1 = word_count_dict[word_1]
# prob_word_2 = word_count_dict[word_2]

# normalized_prob = prob_word_1 / (prob_word_1 + prob_word_2)
# print(normalized_prob)

In [10]:
for key in keyword_pair_dict:
    word_1 = keyword_pair_dict[key]["keywords"][0]
    word_2 = keyword_pair_dict[key]["keywords"][1]
    prob_word_1 = word_count_dict[word_1]
    prob_word_2 = word_count_dict[word_2]
    
    normalized_prob_word_1 = prob_word_1 / (prob_word_1 + prob_word_2)
    
    keyword_pair_dict[key]["probability"] = normalized_prob_word_1

In [11]:
def generate_submission_unigram():
    test_text = readfile('test.txt')

    test_sentences = test_text.split("\n");
    test_word_choices = []
    answer_list = ["Id,Expected"]
    running_number = 1
    for sentence in test_sentences:
        start_index = sentence.find("{")
        end_index = sentence.find("}")

        word_choice = sentence[start_index+1:end_index]
        if(word_choice):
            prob = keyword_pair_dict[word_choice]["probability"]
            answer_list.append("{},{}".format(running_number, prob))
        running_number += 1
    answer = "\n".join(answer_list)
    return answer

In [12]:
# submission = generate_submission_unigram()
# writefile('submission7.csv', submission)

In [13]:
def calculate_bigram_prob(sentence):
    prob_word_choice_1 = 0
    
    start_index = sentence.find("{")
    end_index = sentence.find("}")
    
    word_choice = sentence[(start_index+1):end_index] # "thoir|thóir"

    if word_choice:
        [word_choice_1, word_choice_2] = word_choice.split('|'); # word_choice_1 = "thoir" and word_choice_2 = "thóir"
    #     print(word_choice_1)
    #     print(word_choice_2)

        words = sentence.split();
        index = words.index("{" + word_choice + "}")
        if index == 0:
            prev_word = "^"
            next_word = words[1]

        elif index == len(words)-1:
            prev_word = words[index - 1]
            next_word = "$"

        else:
            prev_word = words[index - 1]
            next_word = words[index + 1]

        # E.g. Tá Dora agus Bróigín ar {thoir|thóir} réalt-ainmhithe in éineacht le Pegaso.
        # p1 = P(thoir|ar) * P(réaltainmhithe|thoir) 
        # c1 = count("ar thoir") * counbt("thoir réaltainmhithe")

        # p2 = P(thòir|ar) * P(réaltainmhithe|thòir)
        # c2 = count("ar thòir") * count("thòir réaltainmhithe")
        # P(thoir) = p1/(p1+p2)
        # P(thoir) = c1/(c1+c2)

        word_1_bigram_1 = prev_word + " " + word_choice_1
        word_1_bigram_2 = word_choice_1 + " " + next_word

        word_2_bigram_1 = prev_word + " " + word_choice_2
        word_2_bigram_2 = word_choice_2 + " " + next_word

        count_word_1_bigram_1 = word_count_dict[word_1_bigram_1] if word_1_bigram_1 in word_count_dict else 1
        count_word_1_bigram_2 = word_count_dict[word_1_bigram_2] if word_1_bigram_2 in word_count_dict else 1

        count_word_2_bigram_1 = word_count_dict[word_2_bigram_1] if word_2_bigram_1 in word_count_dict else 1
        count_word_2_bigram_2 = word_count_dict[word_2_bigram_2] if word_2_bigram_2 in word_count_dict else 1

        c_1 = count_word_1_bigram_1 * count_word_1_bigram_2 
        c_2 = count_word_2_bigram_1 * count_word_2_bigram_2
        prob_word_choice_1 = c_1 / (c_1 + c_2) if c_1 + c_2 > 0 else keyword_pair_dict[word_choice]["probability"]

    return prob_word_choice_1

In [14]:
def generate_submission_bigram():
    test_text = readfile('test.txt')

    test_sentences = test_text.split("\n");
    test_word_choices = []
    answer_list = ["Id,Expected"]
    running_number = 1
    for test_sentence in test_sentences:
        if test_sentence:
            prob = calculate_bigram_prob(test_sentence)

            answer_list.append("{},{}".format(running_number, prob))
            running_number += 1
    answer = "\n".join(answer_list)
    return answer

In [15]:
# print(generate_submission_bigram())

writefile('submission_bigram_1.csv', generate_submission_bigram())